<a href="https://colab.research.google.com/github/homahm/GEM-Benchmark/blob/master/regenerate_input_model_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pickle
import random
from scipy import sparse
import itertools
from scipy.io import savemat, loadmat
import string
import os
import pandas as pd
import random
# Maximum / minimum document frequency
max_df = 0.8
min_df = 100  # choose desired value for min_df

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [73]:
path = "/content/gdrive/MyDrive/Penn/data/TV/"
df = pd.read_csv( path + 'only_fox_2012_2021.csv.gz' ) 
os.listdir(path)

['w2vd300_all_v1.model.wv.vectors.npy',
 'w2vd300_all_v1.model.trainables.syn1.npy',
 'w2vd300_all_v1.model.trainables.syn1neg.npy',
 'w2vd300_all_v1.model',
 'only_cnn_2012_2021.csv.gz',
 'only_fox_2012_2021.csv.gz',
 'scripts',
 'cnn_v1',
 'results_v0',
 'results',
 'fox_v1']

In [74]:
df.head(2)
len(df)

57000

In [75]:
df = df.sort_values(by=['programutcdate'])
df = df.reset_index()
df.head(2)

,index,programutcdate,programinfoduration,programinfotitle,programinfostation,schedulelocaldatetime,transcript,programgenres,transcripturl,gun,immigrant,vaccin,abortion,muslim,processed,ads_free,ads
0,56601,2012-12-01,3600.0,Fox and Friends Saturday,FNC,1.354342e+09,that is all the time we have left. Thank you f...,"[{programgenreid=3894015, programid=5757134, r...",http://mms.tveyes.com/transcript.asp?StationID...,0.0,0.0,0.0,0.0,0.0,that is all the time we have left. thank you f...,"good morning, it's saturday, december 1st. i'm...",that is all the time we have left. thank you f...
1,56580,2012-12-01,3600.0,The O'Reilly Factor,FNC,1.354334e+09,>> I didn't give you anything so I have nothin...,"[{programgenreid=1844749, programid=925639, ro...",http://mms.tveyes.com/transcript.asp?StationID...,1.0,0.0,0.0,0.0,0.0,>> i didn't give you anything so i have nothin...,>> i didn't give you anything so i have nothin...,NaN


In [76]:
inds = list(range(0,len(df)))
random.shuffle(inds)
df = df.iloc[inds[:30000],:]

In [10]:
df.keys()

Index(['index', 'programutcdate', 'programinfoduration', 'programinfotitle',
       'programinfostation', 'schedulelocaldatetime', 'transcript',
       'programgenres', 'transcripturl', 'gun', 'immigrant', 'vaccin',
       'abortion', 'muslim', 'processed', 'ads_free', 'ads'],
      dtype='object')

In [77]:
def remove_not_printable(in_str):
    return "".join([c for c in in_str if c in string.printable])

In [78]:
df = df[~df.ads_free.isnull()]

In [28]:
txt = df.iloc[6].transcript

In [30]:
df[df.transcript==txt]

,index,programutcdate,programinfoduration,programinfotitle,programinfostation,schedulelocaldatetime,transcript,programgenres,transcripturl,gun,immigrant,vaccin,abortion,muslim,processed,ads_free,ads
6,6,2012-12-01,3600.0,Fox and Friends Saturday,FNC,1.354345e+09,"bruising, bleeding, or paleness. If you've had...","[{programgenreid=3894015, programid=5757134, r...",http://mms.tveyes.com/transcript.asp?StationID...,0.0,0.0,0.0,0.0,0.0,"bruising, bleeding, or paleness. if you've had...","bruising, bleeding, or paleness. if you've had...","[ male announcer ] in blind taste tests, prego..."


In [ ]:
df[df.duplicated(subset=['programutcdate', 'programinfoduration', 'programinfotitle','transcript'], keep='first')]


In [79]:
df['index'] = list(range(len(df)))

In [80]:

def clean_up(doc):
    doc = doc.lower().replace('\n', ' ').replace("’", " ").replace("'", " ").translate(str.maketrans(string.punctuation + "0123456789", ' '*len(string.punctuation + "0123456789"))).split()
    doc = [remove_not_printable(w) for w in doc if len(w)>1]
    if len(doc) > 1:
        doc = " ".join(doc)
    return doc

df['text_cleaned'] = df.ads_free.apply(clean_up)

In [81]:
df.head(2)

,index,programutcdate,programinfoduration,programinfotitle,programinfostation,schedulelocaldatetime,transcript,programgenres,transcripturl,gun,immigrant,vaccin,abortion,muslim,processed,ads_free,ads,text_cleaned
39067,0,2018-04-27,3600.0,Fox News @ Night With Shannon Bream,FNC,1.524784e+09,talking about alfie evans. An italian state li...,"[{programgenreid=52376956, programid=34090716,...",http://mms.tveyes.com/transcript.asp?StationID...,0.0,0.0,0.0,0.0,1.0,talking about alfie evans. an italian state li...,talking about alfie evans. an italian state li...,i no longer live with the uncertainties of hep...,talking about alfie evans an italian state lik...
32253,1,2017-04-24,3600.0,Happening Now,FNC,1.493039e+09,former president barack obama I saw on twitter...,"[{programgenreid=5102865, programid=7336948, r...",http://mms.tveyes.com/transcript.asp?StationID...,0.0,4.0,0.0,0.0,0.0,former president barack obama i saw on twitter...,former president barack obama i saw on twitter...,tomorrow is not a given. but entresto is a med...,former president barack obama saw on twitter t...


In [82]:
df['month'] = df.programutcdate.apply(lambda x: x[:7])
df = df.sort_values(by=['month'])
df['month']

286      2012-12
244      2012-12
245      2012-12
413      2012-12
269      2012-12
          ...   
56570    2021-04
56531    2021-04
56562    2021-04
56600    2021-05
56601    2021-05
Name: month, Length: 29800, dtype: object

In [83]:
df['year'] = df.programutcdate.apply(lambda x: x[:4])
df = df.sort_values(by=['year'])
df['year']

286      2012
22       2012
127      2012
172      2012
141      2012
         ... 
55691    2021
55961    2021
55900    2021
55790    2021
56601    2021
Name: year, Length: 29800, dtype: object

In [84]:
timestamps = list(df.year.values) 
docs = list(df.text_cleaned.values)

In [85]:
# Read stopwords
with open( path + './scripts/stops.txt', 'r') as f:
    stops = f.read().split('\n')

In [86]:
# Create count vectorizer
print('counting document frequency of words...')
cvectorizer = CountVectorizer(min_df=min_df, max_df=max_df, stop_words=None)
cvz = cvectorizer.fit_transform(docs).sign()


counting document frequency of words...


In [87]:
# Get vocabulary
print('building the vocabulary...')
sum_counts = cvz.sum(axis=0)
v_size = sum_counts.shape[1]
sum_counts_np = np.zeros(v_size, dtype=int)
for v in range(v_size):
    sum_counts_np[v] = sum_counts[0,v]
word2id = dict([(w, cvectorizer.vocabulary_.get(w)) for w in cvectorizer.vocabulary_])
id2word = dict([(cvectorizer.vocabulary_.get(w), w) for w in cvectorizer.vocabulary_])
del cvectorizer
print('  initial vocabulary size: {}'.format(v_size))


building the vocabulary...
  initial vocabulary size: 22802


In [ ]:
word2id

In [88]:
# Sort elements in vocabulary
idx_sort = np.argsort(sum_counts_np)
vocab_aux = [id2word[idx_sort[cc]] for cc in range(v_size)]

# Filter out stopwords (if any)
vocab_aux = [w for w in vocab_aux if w not in stops]
print('  vocabulary size after removing stopwords from list: {}'.format(len(vocab_aux)))

  vocabulary size after removing stopwords from list: 22514


In [89]:
# Create dictionary and inverse dictionary
vocab = vocab_aux
del vocab_aux
word2id = dict([(w, j) for j, w in enumerate(vocab)])
id2word = dict([(j, w) for j, w in enumerate(vocab)])

# Create mapping of timestamps
all_times = sorted(set(timestamps))
time2id = dict([(t, i) for i, t in enumerate(all_times)])
id2time = dict([(i, t) for i, t in enumerate(all_times)])
time_list = [id2time[i] for i in range(len(all_times))]


In [41]:
len(time_list)

10

In [90]:
# Split in train/test/valid
print('tokenizing documents and splitting into train/test/valid...')
num_docs = cvz.shape[0]
trSize = int(np.floor(0.85*num_docs))
tsSize = int(np.floor(0.10*num_docs))
vaSize = int(num_docs - trSize - tsSize)
del cvz
idx_permute = np.random.permutation(num_docs).astype(int)

tokenizing documents and splitting into train/test/valid...


In [91]:
# Remove words not in train_data
vocab = list(set([w for idx_d in range(trSize) for w in docs[idx_permute[idx_d]].split() if w in word2id]))
word2id = dict([(w, j) for j, w in enumerate(vocab)])
id2word = dict([(j, w) for j, w in enumerate(vocab)])
print('  vocabulary after removing words not in train: {}'.format(len(vocab)))

  vocabulary after removing words not in train: 22514


In [92]:
docs_tr = [[word2id[w] for w in docs[idx_permute[idx_d]].split() if w in word2id] for idx_d in range(trSize)]
timestamps_tr = [time2id[timestamps[idx_permute[idx_d]]] for idx_d in range(trSize)]
docs_ts = [[word2id[w] for w in docs[idx_permute[idx_d+trSize]].split() if w in word2id] for idx_d in range(tsSize)]
timestamps_ts = [time2id[timestamps[idx_permute[idx_d+trSize]]] for idx_d in range(tsSize)]
docs_va = [[word2id[w] for w in docs[idx_permute[idx_d+trSize+tsSize]].split() if w in word2id] for idx_d in range(vaSize)]
timestamps_va = [time2id[timestamps[idx_permute[idx_d+trSize+tsSize]]] for idx_d in range(vaSize)]


In [ ]:
timestamps_tr

In [93]:
print('  number of documents (train): {} [this should be equal to {} and {}]'.format(len(docs_tr), trSize, len(timestamps_tr)))
print('  number of documents (test): {} [this should be equal to {} and {}]'.format(len(docs_ts), tsSize, len(timestamps_ts)))
print('  number of documents (valid): {} [this should be equal to {} and {}]'.format(len(docs_va), vaSize, len(timestamps_va)))


  number of documents (train): 25330 [this should be equal to 25330 and 25330]
  number of documents (test): 2980 [this should be equal to 2980 and 2980]
  number of documents (valid): 1490 [this should be equal to 1490 and 1490]


In [94]:
# Remove empty documents
print('removing empty documents...')

def remove_empty(in_docs, in_timestamps):
    out_docs = []
    out_timestamps = []
    for ii, doc in enumerate(in_docs):
        if(doc!=[]):
            out_docs.append(doc)
            out_timestamps.append(in_timestamps[ii])
    return out_docs, out_timestamps

def remove_by_threshold(in_docs, in_timestamps, thr):
    out_docs = []
    out_timestamps = []
    for ii, doc in enumerate(in_docs):
        if(len(doc)>thr):
            out_docs.append(doc)
            out_timestamps.append(in_timestamps[ii])
    return out_docs, out_timestamps

docs_tr, timestamps_tr = remove_empty(docs_tr, timestamps_tr)
docs_ts, timestamps_ts = remove_empty(docs_ts, timestamps_ts)
docs_va, timestamps_va = remove_empty(docs_va, timestamps_va)

removing empty documents...


In [ ]:
timestamps_tr

[47,
 86,
 50,
 78,
 74,
 3,
 72,
 15,
 19,
 56,
 79,
 82,
 80,
 77,
 28,
 46,
 64,
 47,
 61,
 36,
 22,
 55,
 71,
 7,
 88,
 56,
 71,
 2,
 23,
 68,
 39,
 1,
 42,
 14,
 89,
 50,
 19,
 46,
 37,
 68,
 61,
 41,
 50,
 65,
 34,
 80,
 73,
 25,
 27,
 66,
 50,
 50,
 50,
 76,
 31,
 4,
 73,
 67,
 97,
 96,
 54,
 48,
 6,
 7,
 64,
 85,
 61,
 77,
 19,
 78,
 79,
 80,
 73,
 79,
 67,
 51,
 29,
 50,
 74,
 31,
 74,
 19,
 50,
 24,
 76,
 44,
 54,
 50,
 26,
 43,
 86,
 31,
 59,
 44,
 80,
 5,
 32,
 44,
 2,
 66,
 36,
 63,
 71,
 31,
 50,
 14,
 71,
 73,
 44,
 61,
 44,
 69,
 39,
 37,
 76,
 31,
 67,
 77,
 7,
 66,
 13,
 6,
 23,
 2,
 52,
 18,
 70,
 55,
 53,
 74,
 96,
 40,
 14,
 24,
 51,
 39,
 67,
 77,
 34,
 75,
 50,
 90,
 79,
 48,
 79,
 78,
 3,
 79,
 51,
 44,
 57,
 73,
 78,
 44,
 61,
 70,
 96,
 6,
 31,
 38,
 24,
 89,
 34,
 38,
 44,
 77,
 35,
 66,
 89,
 77,
 4,
 57,
 23,
 53,
 19,
 29,
 31,
 86,
 67,
 33,
 31,
 22,
 75,
 61,
 57,
 64,
 62,
 48,
 23,
 80,
 80,
 16,
 5,
 28,
 23,
 4,
 43,
 35,
 75,
 44,
 56,
 75,
 50,
 4

In [95]:
# Remove test documents with length=1
docs_ts, timestamps_ts = remove_by_threshold(docs_ts, timestamps_ts, 1)

# Split test set in 2 halves
print('splitting test documents in 2 halves...')
docs_ts_h1 = [[w for i,w in enumerate(doc) if i<=len(doc)/2.0-1] for doc in docs_ts]
docs_ts_h2 = [[w for i,w in enumerate(doc) if i>len(doc)/2.0-1] for doc in docs_ts]


splitting test documents in 2 halves...


In [96]:
# Getting lists of words and doc_indices
print('creating lists of words...')

def create_list_words(in_docs):
    return [x for y in in_docs for x in y]

words_tr = create_list_words(docs_tr)
words_ts = create_list_words(docs_ts)
words_ts_h1 = create_list_words(docs_ts_h1)
words_ts_h2 = create_list_words(docs_ts_h2)
words_va = create_list_words(docs_va)

print('  len(words_tr): ', len(words_tr))
print('  len(words_ts): ', len(words_ts))
print('  len(words_ts_h1): ', len(words_ts_h1))
print('  len(words_ts_h2): ', len(words_ts_h2))
print('  len(words_va): ', len(words_va))

creating lists of words...
  len(words_tr):  53577184
  len(words_ts):  6301233
  len(words_ts_h1):  3149861
  len(words_ts_h2):  3151372
  len(words_va):  3217326


In [97]:
# Get doc indices
print('getting doc indices...')

def create_doc_indices(in_docs):
    aux = [[j for i in range(len(doc))] for j, doc in enumerate(in_docs)]
    return [int(x) for y in aux for x in y]

doc_indices_tr = create_doc_indices(docs_tr)
doc_indices_ts = create_doc_indices(docs_ts)
doc_indices_ts_h1 = create_doc_indices(docs_ts_h1)
doc_indices_ts_h2 = create_doc_indices(docs_ts_h2)
doc_indices_va = create_doc_indices(docs_va)

print('  len(np.unique(doc_indices_tr)): {} [this should be {}]'.format(len(np.unique(doc_indices_tr)), len(docs_tr)))
print('  len(np.unique(doc_indices_ts)): {} [this should be {}]'.format(len(np.unique(doc_indices_ts)), len(docs_ts)))
print('  len(np.unique(doc_indices_ts_h1)): {} [this should be {}]'.format(len(np.unique(doc_indices_ts_h1)), len(docs_ts_h1)))
print('  len(np.unique(doc_indices_ts_h2)): {} [this should be {}]'.format(len(np.unique(doc_indices_ts_h2)), len(docs_ts_h2)))
print('  len(np.unique(doc_indices_va)): {} [this should be {}]'.format(len(np.unique(doc_indices_va)), len(docs_va)))


getting doc indices...
  len(np.unique(doc_indices_tr)): 25330 [this should be 25330]
  len(np.unique(doc_indices_ts)): 2980 [this should be 2980]
  len(np.unique(doc_indices_ts_h1)): 2980 [this should be 2980]
  len(np.unique(doc_indices_ts_h2)): 2980 [this should be 2980]
  len(np.unique(doc_indices_va)): 1490 [this should be 1490]


In [98]:
# Number of documents in each set
n_docs_tr = len(docs_tr)
n_docs_ts = len(docs_ts)
n_docs_ts_h1 = len(docs_ts_h1)
n_docs_ts_h2 = len(docs_ts_h2)
n_docs_va = len(docs_va)

# Remove unused variables
del docs_tr
del docs_ts
del docs_ts_h1
del docs_ts_h2
del docs_va

In [99]:
# Create bow representation
print('creating bow representation...')

def create_bow(doc_indices, words, n_docs, vocab_size):
    return sparse.coo_matrix(([1]*len(doc_indices),(doc_indices, words)), shape=(n_docs, vocab_size)).tocsr()

bow_tr = create_bow(doc_indices_tr, words_tr, n_docs_tr, len(vocab))
bow_ts = create_bow(doc_indices_ts, words_ts, n_docs_ts, len(vocab))
bow_ts_h1 = create_bow(doc_indices_ts_h1, words_ts_h1, n_docs_ts_h1, len(vocab))
bow_ts_h2 = create_bow(doc_indices_ts_h2, words_ts_h2, n_docs_ts_h2, len(vocab))
bow_va = create_bow(doc_indices_va, words_va, n_docs_va, len(vocab))

del words_tr
del words_ts
del words_ts_h1
del words_ts_h2
del words_va
del doc_indices_tr
del doc_indices_ts
del doc_indices_ts_h1
del doc_indices_ts_h2
del doc_indices_va

creating bow representation...


In [ ]:
# skip writing files for LDA

In [100]:
path_save = path + "/fox_v1/min_df_100/"

In [101]:
# Also write the vocabulary and timestamps
with open(path_save + 'vocab.txt', "w") as f:
    for v in vocab:
        f.write(v + '\n')

with open(path_save + 'timestamps.txt', "w") as f:
    for t in time_list:
        f.write(t + '\n')

with open(path_save + 'vocab.pkl', 'wb') as f:
    pickle.dump(vocab, f)
del vocab

with open(path_save + 'timestamps.pkl', 'wb') as f:
    pickle.dump(time_list, f)

In [102]:
# Save timestamps alone
savemat(path_save + 'bow_tr_timestamps.mat', {'timestamps': timestamps_tr}, do_compression=True)
savemat(path_save + 'bow_ts_timestamps.mat', {'timestamps': timestamps_ts}, do_compression=True)
savemat(path_save + 'bow_va_timestamps.mat', {'timestamps': timestamps_va}, do_compression=True)

In [103]:
# Split bow intro token/value pairs
print('splitting bow intro token/value pairs and saving to disk...')

def split_bow(bow_in, n_docs):
    indices = [[w for w in bow_in[doc,:].indices] for doc in range(n_docs)]
    counts = [[c for c in bow_in[doc,:].data] for doc in range(n_docs)]
    return indices, counts

bow_tr_tokens, bow_tr_counts = split_bow(bow_tr, n_docs_tr)
savemat(path_save + 'bow_tr_tokens.mat', {'tokens': bow_tr_tokens}, do_compression=True)
savemat(path_save + 'bow_tr_counts.mat', {'counts': bow_tr_counts}, do_compression=True)
del bow_tr
del bow_tr_tokens
del bow_tr_counts

splitting bow intro token/value pairs and saving to disk...


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [104]:
bow_ts_tokens, bow_ts_counts = split_bow(bow_ts, n_docs_ts)
savemat(path_save + 'bow_ts_tokens.mat', {'tokens': bow_ts_tokens}, do_compression=True)
savemat(path_save + 'bow_ts_counts.mat', {'counts': bow_ts_counts}, do_compression=True)
del bow_ts
del bow_ts_tokens
del bow_ts_counts

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [105]:
bow_ts_h1_tokens, bow_ts_h1_counts = split_bow(bow_ts_h1, n_docs_ts_h1)
savemat(path_save + 'bow_ts_h1_tokens.mat', {'tokens': bow_ts_h1_tokens}, do_compression=True)
savemat(path_save + 'bow_ts_h1_counts.mat', {'counts': bow_ts_h1_counts}, do_compression=True)
del bow_ts_h1
del bow_ts_h1_tokens
del bow_ts_h1_counts

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [106]:
bow_ts_h2_tokens, bow_ts_h2_counts = split_bow(bow_ts_h2, n_docs_ts_h2)
savemat(path_save + 'bow_ts_h2_tokens.mat', {'tokens': bow_ts_h2_tokens}, do_compression=True)
savemat(path_save + 'bow_ts_h2_counts.mat', {'counts': bow_ts_h2_counts}, do_compression=True)
del bow_ts_h2
del bow_ts_h2_tokens
del bow_ts_h2_counts

bow_va_tokens, bow_va_counts = split_bow(bow_va, n_docs_va)
savemat(path_save + 'bow_va_tokens.mat', {'tokens': bow_va_tokens}, do_compression=True)
savemat(path_save + 'bow_va_counts.mat', {'counts': bow_va_counts}, do_compression=True)
del bow_va
del bow_va_tokens
del bow_va_counts

print('Data ready !!')
print('*************')

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


Data ready !!
*************


In [108]:
os.chdir('/content/gdrive/MyDrive/Colab Notebooks/DTM/')

In [109]:
!python main.py --dataset fox_v1 --data_path '/content/gdrive/MyDrive/Penn/data/TV/fox_v1' --emb_path 'tv' --min_df 100 --num_topics 50 --lr 0.0001 --epochs 1000 --mode train

Streaming output truncated to the last 5000 lines.
Topic 46 .. Time: 0 ===> ['unwelcome', 'balmy', 'reunification', 'surfers', 'collisions', 'playful', 'snows', 'medicinal', 'buzzed', 'skier', 'simulate', 'hr', 'beams', 'cancun', 'surgeries', 'narcotic', 'anchorage', 'lice', 'reunite']
Topic 46 .. Time: 4 ===> ['archaic', 'pineapple', 'caleb', 'turnpike', 'narcotic', 'xbox', 'consumes', 'suvs', 'vanishing', 'outrun', 'homophobia', 'oat', 'buns', 'intrepid', 'fy', 'labrador', 'bes', 'pom', 'sfx']
Topic 46 .. Time: 8 ===> ['talkshow', 'trumpism', 'purging', 'esdnygov', 'unifier', 'rescinding', 'rancor', 'bastille', 'hans', 'vitriolic', 'reinvigorate', 'centric', 'modernization', 'holdouts', 'ec', 'absently', 'quickbooks', 'preemptively', 'riddance']
Topic 47 .. Time: 0 ===> ['school', 'children', 'connecticut', 'christmas', 'newtown', 'kids', 'police', 'shooting', 'parents', 'family', 'sandy', 'gun', 'mother', 'shooter', 'hook', 'tragedy', 'victims', 'lanza', 'elementary']
Topic 47 .. Ti

In [102]:
import gensim 
model = gensim.models.Word2Vec.load("/content/gdrive/MyDrive/Penn/data/TV/w2vd300_all_v1.model")

In [68]:
import torch
torch.cuda.is_available()

True

In [5]:
os.listdir('/content/gdrive/MyDrive/')
os.chdir('/content/gdrive/MyDrive/Colab Notebooks/DTM/')
os.listdir()

['regenerate input model v1.ipynb',
 'data.py',
 'plot_word_evolution.py',
 'utils.py',
 'regenerate input model.ipynb',
 '__pycache__',
 'results',
 'word_evolution.png',
 'detm.py',
 'main.py']

In [21]:
!python plot_word_evolution.py

beta:  (50, 10, 22514)
timelist:  ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']
ticks:  ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']
Topic 0 .. Time: 0 ===> ['coronavirus', 'laborers', 'couric', 'cindy', 'therapeutics', 'filtered', 'powell', 'blazes', 'cunningham']
Topic 0 .. Time: 1 ===> ['coronavirus', 'couric', 'laborers', 'powell', 'tucker', 'brien', 'suppressed', 'handicapped', 'cunningham']
Topic 0 .. Time: 2 ===> ['coronavirus', 'tucker', 'black', 'african', 'justice', 'powell', 'mail', 'suppress', 'citizenry']
Topic 0 .. Time: 3 ===> ['african', 'black', 'tucker', 'justice', 'mail', 'mark', 'haiti', 'trump', 'depress']
Topic 0 .. Time: 4 ===> ['mail', 'justice', 'african', 'tucker', 'haiti', 'black', 'trump', 'mark', 'judge']
Topic 0 .. Time: 5 ===> ['trump', 'mail', 'donald', 'pete', 'tucker', 'carolina', 'mark', 'martha', 'justice']
Topic 0 .. Time: 6 ===> ['pete', 'trump', 'green', 'georgia', 'donald', 't

In [9]:
os.listdir('/content/gdrive/MyDrive/Penn/data/TV/results_v0')

['detm_cnn_v1_K_3_Htheta_800_Optim_adam_Clip_0.0_ThetaAct_relu_Lr_0.0001_Bsz_1000_RhoSize_300_L_3_minDF_20_trainEmbeddings_1',
 'detm_cnn_v1_K_3_Htheta_800_Optim_adam_Clip_0.0_ThetaAct_relu_Lr_0.0001_Bsz_1000_RhoSize_300_L_3_minDF_20_trainEmbeddings_1_beta.mat',
 'detm_cnn_v1_K_3_Htheta_800_Optim_adam_Clip_0.0_ThetaAct_relu_Lr_0.0001_Bsz_1000_RhoSize_300_L_3_minDF_20_trainEmbeddings_1_rho.mat',
 'detm_cnn_v1_K_30_Htheta_800_Optim_adam_Clip_0.0_ThetaAct_relu_Lr_0.0001_Bsz_1000_RhoSize_300_L_3_minDF_20_trainEmbeddings_1',
 'detm_cnn_v1_K_30_Htheta_800_Optim_adam_Clip_0.0_ThetaAct_relu_Lr_0.0001_Bsz_1000_RhoSize_300_L_3_minDF_20_trainEmbeddings_1_beta.mat',
 'detm_cnn_v1_K_30_Htheta_800_Optim_adam_Clip_0.0_ThetaAct_relu_Lr_0.0001_Bsz_1000_RhoSize_300_L_3_minDF_20_trainEmbeddings_1_rho.mat']